# (Pseudo-) Online Feature Calculation

This notebook is used to receive data from the HoloLens 2. The data is chunked in 10 second pieces.

In [ ]:
from flask import Flask, request
import json
import requests
import os
import pandas as pd

## Read the data from the newly arrived csv-file and call the feature calculation

## Run FeaturesCalculation notebook to make its function accessible here

In [ ]:
%run -i FeaturesCalculation.ipynb

## Calculate the features and save them as csv
See `FeaturesCalculation.ipynb` for more details.

In [ ]:
def calculate_features_for_10s_chunk(newdf):
    list_of_features = []
    newdf_valid = only_valid_data(newdf)
    df_fixations = get_fixation_df(newdf_valid)
    features = calculate_fixation_features(df_fixations, 10)
    blinks = calculate_blink_features(newdf,10)
    directions = calculate_directions_of_list(df_fixations) 
    density = calculate_fixation_density(newdf_valid, df_fixations)
    features.update(blinks)
    features.update(directions)
    features.update(density)
    features["label"] = ""
    features["duration"] = "10"
    # change the participant ID here, when the participant changes
    features["participant_id"] = "001"            
    list_of_features.append(features)  
    flat_ls = [item for sublist in list_of_features for item in sublist]
    # change the participant ID here, when the participant changes
    feature_file_path = save_as_csv(list_of_features, "001", './OnlineFeatureFiles/')
    return feature_file_path

In [ ]:
def csv_to_features(gaze_data_file_path):
    # continue only if the csv-file contains data
    if os.stat(gaze_data_file_path).st_size != 0:
        df = pd.read_csv(gaze_data_file_path)
        feature_file_path = calculate_features_for_10s_chunk(df)
        print(f"Feature calculation done for: {gaze_data_file_path}")
        print(f"Feature file path: {feature_file_path}")
        return feature_file_path
# csv_to_features("./HL2_DataCollection/2022_09_23-13_41_19-Alex01-Inspection02.csv")

## Run SVM notebook to make its function accessible here
If you are not training on normalized data this make take a while.

In [ ]:
%run -i SVM_Test_On_HL2_Data_.ipynb

## Predict the class for the last arrived data chunk

In [ ]:
def normalize_values(df): 
    scaler = MaxAbsScaler()
    training_features_plus_new_row = pd.concat([features, df])  # features -> contains all the features for training and testing
    scaler.fit(training_features_plus_new_row)
    scaled = scaler.transform(training_features_plus_new_row)
    scaled_features = pd.DataFrame(scaled, columns=df.columns)
    row_for_last_chunk = scaled_features.tail(1)
    print("Normalized Features:")
    display(row_for_last_chunk)
    return row_for_last_chunk

In [ ]:
row_counter = 0
classes = ["Inspection", "Reading", "Search"]
kernels = [linear, poly, rbf, sig]
def predict_class_for_last_chunk(feature_file_path):
    if os.stat(feature_file_path).st_size != 0:
        df = pd.read_csv(feature_file_path)
        cur_number_of_rows = df.shape[0]
        # only continue if we have a new row
        if cur_number_of_rows > row_counter:
            last_row = df.tail(1)
            display(last_row)
            
            lr_features= last_row[["meanFix", "maxFix", "varFix", "xDir", "yDir", "fixDensPerBB"]]
            last_row = normalize_values(lr_features)
            
            classes = linear.classes_
            print(f"\n---\nclasses: {classes}")
            
            results = {}
            
            for kernel in kernels:
                new_pred_prob = kernel.predict_proba(last_row)
                new_pred_class = kernel.predict(last_row)
                # print("\n---\nNew Prediction:")
                display(new_pred_prob)
                display(new_pred_class)
                pred_list = new_pred_prob.tolist()[0]
                max_prob = max(pred_list)
                # print(f"max_prob: {max_prob}")
                max_index = pred_list.index(max_prob)
                # print(f"max_index: {max_prob}")
                results[kernel.kernel] = {"pred_class": new_pred_class[0], "max_prob": max_prob, "max_index": max_index }
                # results[kernel.kernel] = [new_pred_class[0], max_prob, max_index ]
                
            display(results)
        
            
            max_proba = {max(float(d['max_prob']) for d in results.values())}
            key = [i for i in results if results[i]['max_prob']==max_proba]
            # print(f"max_proba: {max_proba}, key: {key}")
            predicts = [d['pred_class'] for d in results.values()]
            final_class_from_predict = max(set(predicts), key = predicts.count)
            # print(f"final_class_from_predict: {final_class_from_predict}")
            
            
            new_linear_pred_prob = linear.predict_proba(last_row)
            new_linear_pred_class = linear.predict(last_row)
            # print("\n---\nNew Linear Prediction:")
            # display(new_linear_pred_prob)
            # display(new_linear_pred_class)
            lin_list = new_linear_pred_prob.tolist()[0]
            lin_max_prob = max(lin_list)
            # print(f"lin_max_prob: {lin_max_prob}")
            lin_max_index = lin_list.index(lin_max_prob)
            # print(f"lin_max_index: {lin_max_index}")

            new_poly_pred_prob = poly.predict_proba(last_row)
            new_poly_pred_class = poly.predict(last_row)
            # print("\n---\nNew Poly Prediction:")
            # display(new_poly_pred_prob)
            # display(new_poly_pred_class)
            poly_list = new_poly_pred_prob.tolist()[0]
            poly_max_prob = max(poly_list)
            # print(f"poly_max_prob: {poly_max_prob}")
            poly_max_index = poly_list.index(poly_max_prob)
            # print(f"poly_max_index: {poly_max_index}")

            new_rbf_pred_prob = rbf.predict_proba(last_row)
            new_rbf_pred_class = rbf.predict(last_row)
            # print("\n---\nNew RBF Prediction:")
            # display(new_rbf_pred_prob)
            # display(new_rbf_pred_class)
            rbf_list = new_rbf_pred_prob.tolist()[0]
            rbf_max_prob = max(rbf_list)
            # print(f"rbf_max_prob: {rbf_max_prob}")
            rbf_max_index = rbf_list.index(rbf_max_prob)
            # print(f"rbf_max_index: {rbf_max_index}")
            

            new_sig_pred_prob = sig.predict_proba(last_row)
            new_sig_pred_class = sig.predict(last_row)
            # print("\n---\nNew Sig Prediction:")
            # display(new_sig_pred_prob)
            # display(new_sig_pred_class)
            sig_list = new_sig_pred_prob.tolist()[0]
            sig_max_prob = max(sig_list)
            # print(f"sig_max_prob: {sig_max_prob}")
            sig_max_index = sig_list.index(sig_max_prob)
            # print(f"sig_max_index: {sig_max_index}")
            
            print("----------------")
            max_probs = { lin_max_prob: lin_max_index, poly_max_prob: poly_max_index,
                        rbf_max_prob: rbf_max_index}
            
            max_prediction_value = max(max_probs, key=float)
            # print(max_prediction_value)
            final_max_index = max_probs[max_prediction_value]
            final_class_from_probs = classes[final_max_index]
            
            print(f"final_class_from_probs: {final_class_from_probs}")
            
            predicts = [new_linear_pred_class[0], new_poly_pred_class[0], new_rbf_pred_class[0]]
            final_class_from_predict = max(set(predicts), key = predicts.count)
            print(f"final_class_from_predict: {final_class_from_predict}")
            
            
            same_class_predicted = (final_class_from_probs == final_class_from_predict)
            print(f"same_class_predicted: {same_class_predicted}")
            
            if (same_class_predicted and max_prediction_value > 0.5):
                # send request to HL2 with class and probability
                send_activity(final_class_from_probs, max_prediction_value)
   

## Send the recognized activity back to the HooLens 2 

In [ ]:
import urllib.parse

import requests
import random
import time


def send_activity(activity, probability):

    # insert the HoloLens 2's IP address here
    holo_url = "http://0.0.0.0:5000"

    url = "{}/?activity={}&probability={}".format(str(holo_url), str(activity), str(probability))
    print(url)

    try:
        r = requests.get(url, timeout=120)
        print(r)
        if r.status_code == 200:
            print("Notified Hololens about activity {}".format(activity))
        else:
            print("Request {} failed with status code {}".format(url, r.status_code))
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))

    else:
        return

def create_and_send_test_data():
    activities = ["reading", "writing", "searching", "inspecting"]
    a = random.randint(0, 3)
    activity = activities[a]
    confidence = random.uniform(0, 1)
    print(f"{activity}: {confidence}")
    send_activity(activity, confidence)


def sender():
    start_time = time.time()
    interval = 5
    for i in range(20):
        time.sleep(start_time + i * interval - time.time())
        create_and_send_test_data()
        print("sent data")

## Run a simple Flask server that receives the raw gaze data from the HL2
Make sure to enter your computer's IP-address as host and the correct port!

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['POST', 'PUT'])
def result():
    
    new_csv = request.files["gazedata"].read()
    filename = request.form["filename"]
    print(f"filename: {filename}")
    filepath = os.path.join("./OnlineGazeDataChunks/", filename)
    print(f"filepath: {filepath}")
    outF = open(filepath, "wb")
    outF.write(new_csv)
    feature_file_path = csv_to_features(filepath)
    predict_class_for_last_chunk(feature_file_path)
    return 'Received !'  

if __name__ == '__main__':
    # insert the IP address here to which the Unity app sends the gaze data 
    # (i.e. the public address of the computer this file runs on)
    app.run(host='localhost', port=5555)
